<a href="https://colab.research.google.com/github/danriv16/danriv16/blob/main/Deterministic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
!pip install gurobipy

In [62]:
# save as humanitarian_gurobi.py and run: python humanitarian_gurobi.py
from gurobipy import Model, GRB, LinExpr, QuadExpr, quicksum
import itertools

In [63]:
# ---------- Sets ----------
T = [1,2,3] #Time
DC = [1,2,3] #Distribution_Center
DP = [1,2,3,4,5] #Demand_Point: StaLucia, Suan, Campo de la Cruz, Candelaria, Manati
C = [1,2] #Commodity


In [64]:
#--------- Parameters -----
# Initial Demand
Initial_Demand_C_DP = {
  (1,1): 215, (1,2):192, (1,3):331, (1,4):169, (1,5):318,
  (2,1): 235, (2,2):210, (2,3):364, (2,4):186, (2,5):349,
}

Inventory_Cost_C = {1: 3.33, 2: 2.50}

Transportation_Cost_DC_DP = {
  (1,1):64, (1,2):64, (1,3):61, (1,4):67, (1,5):61,
  (2,1):30, (2,2):30, (2,3):30, (2,4):36, (2,5):30,
  (3,1):33, (3,2):33, (3,3):30, (3,4):36, (3,5):33,
}

Initial_Inventory_C_DC = {
  (1,1):460, (1,2):385, (1,3):385,
  (2,1):505, (2,2):420, (2,3):420,
}

Fixed_Cost_Open_DC = {1:7580, 2:3790, 3:3790}

Cost_Commodity = {1:333, 2:250}

# small constant
Epsilon = 0.001

In [65]:
def sum_demand_by_commodity(commodity_index, parameter):
  """Sums the initial demand for a given commodity across all demand points."""
  return sum(v for (c, dp), v in parameter.items() if c == commodity_index)

# Example usage:
print(f"Total demand for commodity 1: {sum_demand_by_commodity(1,Initial_Demand_C_DP)}")
print(f"Total capacity for commodity 1: {sum_demand_by_commodity(1,Initial_Inventory_C_DC)}")


# Example usage:
print(f"Total demand for commodity 2: {sum_demand_by_commodity(2,Initial_Demand_C_DP)}")
print(f"Total capacity for commodity 2: {sum_demand_by_commodity(2,Initial_Inventory_C_DC)}")


Total demand for commodity 1: 1225
Total capacity for commodity 1: 1230
Total demand for commodity 2: 1344
Total capacity for commodity 2: 1345


In [66]:
import math

Deprivation_cost = {
  (1,1):math.exp(1.503e1 + 0.1172 * 1*6) - math.exp(1.5), (1,2):math.exp(1.503e1 + 0.1172 * 2*6) - math.exp(1.5), (1,3):math.exp(1.503e1 + 0.1172 * 3*6) - math.exp(1.5),
  (2,1):math.exp(0.065 * 1*6) - 1, (2,2):math.exp(0.065 * 2*6) - 1, (2,3):math.exp(0.065 * 3*6) - 1,
}


In [67]:

#----------- Variables-------
m = Model('NovaBeverages')

X = m.addVars(C, DC, DP, T,name='Transportation',lb=0,vtype=GRB.INTEGER)
Y = m.addVars(DC,vtype=GRB.BINARY,name='OpenDC')
RD = m.addVars(C, DP, T,name='Remaining_Demand',lb=0,vtype=GRB.INTEGER)
I = m.addVars(C, DC, T,name='Inventory',lb=0,vtype=GRB.INTEGER)


In [68]:
# Total amount of commodities sent to DPs lower or equal to initial inventory
for c in C:
    for i in DC:
        m.addConstr(quicksum(X[c,i,j,t] for j in DP for t in T) <= Initial_Inventory_C_DC[c,i]*Y[i])


In [69]:
# Total amount of commodities sent to DPs lower or equal to initial inventory
for c in C:
    for j in DP:
        m.addConstr(quicksum(X[c,i,j,t] for i in DC for t in T) >= Initial_Demand_C_DP[c,j])

In [70]:
#Inventory of commondity c in DC i at time t
for c in C:
    for i in DC:
        for t in T:
            if t==1:
              m.addConstr( I[c,i,t] == Initial_Inventory_C_DC[c,i]- quicksum(X[c,i,j,t] for j in DP) )
            else:
              m.addConstr( I[c,i,t] == I[c,i,t-1]- quicksum(X[c,i,j,t] for j in DP) )



In [71]:
#Demand of commondity c in DP i at time t
for c in C:
    for j in DP:
        for t in T:
            if t==1:
              m.addConstr(  quicksum(X[c,i,j,t] for i in DC) >= Initial_Demand_C_DP[c,j] )
            else:
              m.addConstr( RD[c,j,t] >= RD[c,j,t-1]- quicksum(X[c,i,j,t] for i in DC) )

In [72]:
#Demand of commondity c in DP i at time t
for c in C:
    for j in DP:
        for t in T:
            if t==1:
              m.addConstr( RD[c,j,t] >= Initial_Demand_C_DP[c,j]- quicksum(X[c,i,j,t] for i in DC) )
            else:
              m.addConstr( RD[c,j,t] >= RD[c,j,t-1]- quicksum(X[c,i,j,t] for i in DC) )

In [73]:
m.setObjective(
    quicksum(Inventory_Cost_C[c]*I[c,i,t] for c in C for i in DC for t in T)
  + quicksum(Transportation_Cost_DC_DP[i,j]*X[c,i,j,t] for c in C for i in DC for j in DP for t in T)
  + quicksum(Fixed_Cost_Open_DC[i]*Y[i] for i in DC)
  + quicksum(Cost_Commodity[c]*X[c,i,j,t] for c in C for i in DC for j in DP for t in T)
  + quicksum(Deprivation_cost[c,t]*(RD[c,j,t]/Initial_Demand_C_DP[c,j]) for c in C for j in DP for t in T),
  GRB.MINIMIZE
)


In [74]:
m.optimize()


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 94 rows, 141 columns and 576 nonzeros
Model fingerprint: 0x49933eed
Variable types: 0 continuous, 141 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e-03, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 5e+02]
Found heuristic solution: objective 871675.00000
Presolve removed 78 rows and 108 columns
Presolve time: 0.00s
Presolved: 16 rows, 33 columns, 66 nonzeros
Found heuristic solution: objective 869724.95000
Variable types: 0 continuous, 33 integer (3 binary)
Found heuristic solution: objective 869421.45000

Root relaxation: objective 8.681974e+05, 20 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     

In [77]:
if m.status == GRB.OPTIMAL:
    print(f'Optimal total cost: {m.objVal:.2f}')
    print('DC opening:')
    for i in DC:
        print(i, int(Y[i].x+0.5))

    print('Transportation:')
    for t in T:
        for i in DC:
            for j in DP:
              for c in C:
                v = X[c,i,j,t].x
                if v>1e-6: print(f'c{c}, i{i}, j{j}, t{t} {v:.0f}')

    print('Unmet Demand:')
    for t in T:
        for j in DC:
            for c in C:
              w = RD[c,j,t].x
              if w>1e-6:print(f'c{c}, j{j}, t{t} {RD[c,j,t].x:.0f}')

    print('Inventory:')
    for t in T:
        for i in DC:
            for c in C:
              h = I[c,i,t].x
              if h>1e-6:print(f'c{c}, j{j}, t{t} {I[c,i,t].x:.0f}')




Optimal total cost: 868212.45
DC opening:
1 1
2 1
3 1
Transportation:
c1, i1, j1, t1 22
c2, i1, j1, t1 25
c2, i1, j3, t1 130
c1, i1, j4, t1 115
c1, i1, j5, t1 318
c2, i1, j5, t1 349
c1, i2, j1, t1 193
c2, i2, j1, t1 210
c1, i2, j2, t1 192
c2, i2, j2, t1 210
c1, i3, j3, t1 331
c2, i3, j3, t1 234
c1, i3, j4, t1 54
c2, i3, j4, t1 186
Unmet Demand:
Inventory:
c1, j3, t1 5
c2, j3, t1 1
c1, j3, t2 5
c2, j3, t2 1
c1, j3, t3 5
c2, j3, t3 1
